<a href="https://colab.research.google.com/github/sergio-cabrales/python/blob/main/Fundamental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Analyzing Financial Statements for Insight into Company's Valuation for Possible Investment
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

symbol = 'EC.NZ'
url = 'https://finance.yahoo.com/quote/EC/balance-sheet?p=EC'

# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above

page = requests.get(url, headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want

tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element

tree.xpath("//h1/text()")

['Ecopetrol S.A. (EC)']

In [4]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.

assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4
0,Breakdown,12/31/2020,12/31/2019,12/31/2018,12/31/2017
1,Total Assets,"139,417,000,000","133,890,296,000","124,643,498,000","117,847,412,000"
2,Total Liabilities Net Minority Interest,"83,598,000,000","75,658,668,000","67,535,718,000","69,631,713,000"
3,Total Equity Gross Minority Interest,"55,819,000,000","58,231,628,000","57,107,780,000","48,215,699,000"
4,Total Capitalization,"93,903,000,000","87,640,756,000","89,180,632,000","84,838,284,000"
5,Common Stock Equity,"52,095,000,000","54,413,790,000","55,137,914,000","46,434,953,000"
6,Net Tangible Assets,"51,540,000,000","52,770,770,000","53,567,245,000","44,894,805,000"
7,Working Capital,"4,552,000,000","1,622,162,000","9,205,990,000","6,377,405,000"
8,Invested Capital,"98,826,000,000","92,652,929,000","93,200,559,000","89,982,788,000"
9,Tangible Book Value,"51,540,000,000","52,770,770,000","53,567,245,000","44,894,805,000"
